In [1]:
from utility import *
from SB_regularization import *
import concurrent.futures

In [2]:
"""
import importlib
#importlib.reload(utility)
importlib.reload(SB_regularization)
"""

'\nimport importlib\n#importlib.reload(utility)\nimportlib.reload(SB_regularization)\n'

In [3]:
# Parameters
input_directory = r"~/data/data.csv"

start_date = '1/3/2005'
end_date = '12/31/2010'

transaction_cost = 0.001

In [ ]:
# Data Cleaning
generate_data = data_generation(input_directory, start_date, end_date)
ln_open_np, open_df, close_df, volume_df, open_return_df, close_return_df, Tickers, SPY_culmulative_return = generate_data.output_data(dropna=True)
open_return_df = open_return_df.iloc[1:].dropna(axis=1)
open_return_df.index = pd.to_datetime(open_return_df.index)

In [ ]:
%%time

MDR_normal_model = MDR_model(open_return_df, regularization=False, rebalance="Q")
MDR_regularized_model = MDR_model(open_return_df, regularization=True, rebalance="Q")
MSR_normal_model = MSR_model(open_return_df, regularization=False, rebalance="Q")
MSR_regularized_model = MSR_model(open_return_df, regularization=True, rebalance="Q")
GMV_normal_model = GMV_model(open_return_df, regularization=False, rebalance="Q")
GMV_regularized_model = GMV_model(open_return_df, regularization=True,rebalance="Q")

with concurrent.futures.ProcessPoolExecutor(max_workers=2) as executor:
    MDR_normal_result = executor.submit(MDR_normal_model.fit)
    MDR_regularized_result = executor.submit(MDR_regularized_model.fit)
    MSR_normal_result = executor.submit(MSR_normal_model.fit)
    MSR_regularized_result = executor.submit(MSR_regularized_model.fit)
    GMV_normal_result = executor.submit(GMV_normal_model.fit)
    GMV_regularized_result = executor.submit(GMV_regularized_model.fit)

MDR_normal_model = MDR_normal_result.result()
MDR_regularized_model = MDR_regularized_result.result()
MSR_normal_model = MSR_normal_result.result()
MSR_regularized_model = MSR_regularized_result.result()
GMV_normal_model = GMV_normal_result.result()
GMV_regularized_model = GMV_regularized_result.result()

MDR_daily_return, MDR_cumulative_return = MDR_normal_model.calc_return()
MDR_daily_return_regularized, MDR_cumulative_return_regularized = MDR_regularized_model.calc_return()
MSR_daily_return, MSR_cumulative_return = MSR_normal_model.calc_return()
MSR_daily_return_regularized, MSR_cumulative_return_regularized = MSR_regularized_model.calc_return()
GMV_daily_return, GMV_cumulative_return = GMV_normal_model.calc_return()
GMV_daily_return_regularized, GMV_cumulative_return_regularized = GMV_regularized_model.calc_return()

MDR:   4%|▍         | 1/24 [00:26<10:13, 26.69s/it]

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(
      x=SPY_culmulative_return.index,
      y=SPY_culmulative_return,
      name="SPY"
  ))

fig.add_trace(go.Scatter(
      x=SPY_culmulative_return.index,
      y=MDR_cumulative_return,
      name="MSR"
  ))

fig.add_trace(go.Scatter(
      x=SPY_culmulative_return.index,
      y=MDR_cumulative_return_regularized,
      name="MSR_regularized"
  ))

fig.show()

In [8]:
fig = go.Figure()

fig.add_trace(go.Scatter(
      x=SPY_culmulative_return.index,
      y=SPY_culmulative_return,
      name="SPY"
  ))

fig.add_trace(go.Scatter(
      x=SPY_culmulative_return.index,
      y=GMV_cumulative_return,
      name="GMV"
  ))

fig.add_trace(go.Scatter(
      x=SPY_culmulative_return.index,
      y=GMV_cumulative_return_regularized,
      name="GMV_regularized"
  ))

fig.show()